# Boosting

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn import tree

In [3]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,regularizar,targetBooleano,reemplazarCategoricas_OHE,reemplazarNullsNum,keepFeat_OHE

[###] Initial Preprocessings Done                           
[###] Aditional Preprocessings Done                                                   


In [4]:
from utilities import score, score2, df_feat, df_targ, train_val

### Codigo a correr

In [5]:
MAX_JOBS = 5

**Run :** Entrenar Nuevamente o Cargar Entrenado

In [6]:
runSimple = True
runValidated = True
runValidated2 = True
runReg = True
runOhe = True

**Save :** Guardar Modelo (pisa anterior)

In [7]:
saveSimple = True
saveValidated = True
saveValidated2 = True
saveReg = True
saveOhe = True
saveBest = True

In [8]:
predictions = pd.DataFrame()

### Preprocesamiento

In [9]:
targetBooleano(df_targ, inplace=True)
ohe_feat = reemplazarCategoricas_OHE(df_feat)

reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)
df_feat_r = regularizar(df_feat)

reemplazarNullsNum(ohe_feat, inplace=True)
reemplazarFechas(ohe_feat , inplace=True)
# keepFeat_OHE(ohe_feat, 10, inplace=True)

,horas_de_sol,humedad_tarde,humedad_temprano,mm_evaporados_agua,mm_lluvia_dia,nubosidad_tarde,nubosidad_temprano,presion_atmosferica_tarde,presion_atmosferica_temprano,rafaga_viento_max_velocidad,...,missing_rafaga_viento_max_velocidad,missing_temp_max,missing_temp_min,missing_temperatura_tarde,missing_temperatura_temprano,missing_velocidad_viendo_tarde,missing_velocidad_viendo_temprano,y,m,d
id,,,,,,,,,,,,,,,,,,,,,
74816,7.629393,78.0,74.0,5.470542,68.0,4.50597,4.430203,1024.200000,1026.200000,48.0,...,False,False,False,False,False,False,False,2014,6,7
64318,7.629393,14.0,56.0,5.470542,5.8,1.00000,4.430203,998.300000,1002.500000,61.0,...,False,False,False,False,False,False,False,2014,12,7
54357,2.600000,79.0,47.0,5.470542,0.0,8.00000,7.000000,1000.500000,1005.800000,57.0,...,False,False,False,False,False,False,False,2010,7,8
1762,0.000000,92.0,96.0,3.000000,5.0,8.00000,8.000000,1015.400000,1016.800000,54.0,...,False,False,False,False,False,False,False,2015,11,4
89422,7.629393,14.0,37.0,5.470542,0.0,2.00000,4.430203,1006.400000,1011.100000,50.0,...,False,False,False,False,False,False,False,2014,12,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49272,6.500000,63.0,44.0,3.800000,0.4,6.00000,2.000000,1011.800000,1014.500000,72.0,...,False,False,False,False,False,False,False,2012,5,23
13885,2.100000,65.0,80.0,1.600000,5.2,7.00000,7.000000,1017.500000,1016.300000,48.0,...,False,False,False,False,False,False,False,2011,10,25
4586,7.629393,81.0,62.0,5.470542,0.0,4.50597,4.430203,1015.257118,1017.655555,35.0,...,False,False,False,False,False,False,False,2015,12,16


In [10]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102356 entries, 74816 to 32535
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   direccion_viento_tarde        102356 non-null  float64
 1   direccion_viento_temprano     102356 non-null  float64
 2   horas_de_sol                  102356 non-null  float64
 3   humedad_tarde                 102356 non-null  float64
 4   humedad_temprano              102356 non-null  float64
 5   llovieron_hamburguesas_hoy    102356 non-null  bool   
 6   mm_evaporados_agua            102356 non-null  float64
 7   mm_lluvia_dia                 102356 non-null  float64
 8   nubosidad_tarde               102356 non-null  float64
 9   nubosidad_temprano            102356 non-null  float64
 10  presion_atmosferica_tarde     102356 non-null  float64
 11  presion_atmosferica_temprano  102356 non-null  float64
 12  rafaga_viento_max_direccion   102356 non-

In [11]:
train_feat, val_feat, train_targ, val_targ = train_val( df_feat, df_targ, True )
train_feat_r, val_feat_r, train_targ_r, val_targ_r = train_val( df_feat_r, df_targ, True )
train_feat_ohe, val_feat_ohe, train_targ_ohe, val_targ_ohe = train_val( ohe_feat, df_targ, True )

## Entrenamiento

### Simple

In [ ]:
if (runSimple):
    boost_simple = GradientBoostingClassifier(
        random_state=1,
        max_depth=3,
        min_samples_split=200,
    )
    boost_simple.fit( train_feat, train_targ )
    
    # Guardar
    if(saveSimple):
        dump(boost_simple, 'models/Ensambles/boost_simple.sk') 
else:
    # Cargar
    boost_simple = load('models/Ensambles/boost_simple.sk')

In [ ]:
%time
pred = boost_simple.predict(val_feat)
prob = boost_simple.predict_proba(val_feat)
score(val_targ, pred, prob[:,1])

In [ ]:
pdf = score2('Simple','Basico',val_targ, pred, prob[:,1])
predictions = predictions.append( pdf )

### Validado

In [ ]:
if( runValidated ):
    boost = GradientBoostingClassifier(
        random_state=1,
    )
    
    params = {
        'max_depth': np.arange(1, 20, 1),
        'min_samples_leaf': np.arange(50, 1000, 50),
        'n_estimators': np.arange(50,500,50),
        'loss': ['deviance', 'exponential'],
        'learning_rate': [0.05,0.1,0.2],
    }
    
    boost_grid = RandomizedSearchCV(
        boost,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1
    ).fit(train_feat, train_targ)
    
    # Guardar
    if(saveValidated):
        dump(boost_grid, 'models/Ensambles/boost_grid.sk')
else:
    # Cargar
    boost_grid = load('models/Ensambles/boost_grid.sk')

In [ ]:
print(f"Best score: {boost_grid.best_score_}")
print(f"Best params {boost_grid.best_params_}")

In [ ]:
boost = boost_grid.best_estimator_

In [ ]:
%%time
pred = boost.predict(val_feat)
prob = boost.predict_proba(val_feat)
score(val_targ, pred, prob[:,1])

In [ ]:
pdf = score2('Val','Basico',val_targ, pred, prob[:,1])
predictions = predictions.append( pdf )

### Optimizando mas hiperparametros

In [ ]:
if( runValidated2 ):
    boost2 = GradientBoostingClassifier(
        random_state=1,
    )
    
    params = {
        'max_depth': np.arange(15, 19, 1),
        'min_samples_leaf': np.arange(50, 200, 50),
        'min_samples_split': np.arange(50, 200, 50),
        'n_estimators': np.arange(200,500,100),
        'loss': ['deviance', 'exponential'],
        'learning_rate': [0.03,0.05,0.07],
        'criterion' :['friedman_mse','squared_error']
    }
    
    boost_grid2 = HalvingGridSearchCV(
        boost2,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1
    ).fit(train_feat, train_targ)
    
    # Guardar
    if(saveValidated2):
        dump(boost_grid2, 'models/Ensambles/boost2_grid.sk')
else:
    # Cargar
    boost_grid2 = load('models/Ensambles/boost2_grid.sk')

In [ ]:
print(f"Best score: {boost_grid2.best_score_}")
print(f"Best params {boost_grid2.best_params_}")

In [ ]:
boost2 = boost_grid2.best_estimator_

In [ ]:
%%time
pred = boost2.predict(val_feat)
prob = boost2.predict_proba(val_feat)
score(val_targ, pred2, prob2[:,1])

In [ ]:
pdf = score2('Opt','Basico',val_targ, pred, prob[:,1])
predictions = predictions.append( pdf )

### Reg

In [ ]:
if( runReg ):
    boost3 = GradientBoostingClassifier(
        random_state=1,
    )
    
    params = {
        'max_depth': np.arange(1, 20, 1),
        'min_samples_leaf': np.arange(50, 1000, 50),
        'n_estimators': np.arange(50,500,50),
        'loss': ['deviance', 'exponential'],
        'learning_rate': [0.05,0.1,0.2],
    }
    
    boost_grid3 = RandomizedSearchCV(
        boost3,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1
    ).fit(train_feat_r, train_targ_r)
    
    # Guardar
    if(saveReg):
        dump(boost_grid3, 'models/Ensambles/boost_grid_r.sk')
else:
    # Cargar
    boost_grid3 = load('models/Ensambles/boost_grid_r.sk')

In [ ]:
print(f"Best score: {boost_grid3.best_score_}")
print(f"Best params {boost_grid3.best_params_}")

In [ ]:
boost3 = boost_grid3.best_estimator_

In [ ]:
%%time
pred = boost3.predict(val_feat_r)
prob = boost3.predict_proba(val_feat_r)
score(val_targ_r, pred3, prob3[:,1])

In [ ]:
pdf = score2('Reg','Regularizado',val_targ_r, pred, prob[:,1])
predictions = predictions.append( pdf )

### One Hot Encoding

In [ ]:
if( runReg ):
    boost4 = GradientBoostingClassifier(
        random_state=1,
    )
    
    params = {
        'max_depth': np.arange(1, 20, 1),
        'min_samples_leaf': np.arange(50, 1000, 50),
        'n_estimators': np.arange(50,500,50),
        'loss': ['deviance', 'exponential'],
        'learning_rate': [0.05,0.1,0.2],
    }
    
    boost_grid4 = RandomizedSearchCV(
        boost4,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1
    ).fit(train_feat_ohe, train_targ_ohe)
    
    # Guardar
    if(saveReg):
        dump(boost_grid4, 'models/Ensambles/boost_grid_ohe.sk')
else:
    # Cargar
    boost_grid4 = load('models/Ensambles/boost_grid_ohe.sk')

In [ ]:
print(f"Best score: {boost_grid4.best_score_}")
print(f"Best params {boost_grid4.best_params_}")

In [ ]:
boost3 = boost_grid3.best_estimator_

In [ ]:
%%time
pred = boost3.predict(val_feat_ohe)
prob = boost3.predict_proba(val_feat_ohe)
score(val_targ_ohe, pred3, prob3[:,1])

In [ ]:
pdf = score2('Reg','Regularizado',val_targ_ohe, pred, prob[:,1])
predictions = predictions.append( pdf )

## Predictions

In [ ]:
predictions

In [ ]:
if(saveBest):
    print("todo")
    #dump(forest_grid.best_estimator_, 'models/Ensambles/random_forest.sk')